# Рекомендательные системы

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

In [1]:
import numpy as np
import pandas as pd

#### Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна 0.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [2]:
train_pay = []
train_not_pay = []
with open("coursera_sessions_train.txt", "r") as f:
    for s in f:
        x = s.rstrip().split(";")
        if not x[1]:
            train_not_pay.append(np.array([int(y) for y in x[0].split(",")]))
        else:
            train_pay.append([np.array([int(y) for y in x[0].split(",")]), np.array([int(y) for y in x[1].split(",")])])

In [3]:
n_prod = max(max(max(x[0].max(), x[1].max()) for x in train_pay), max([x.max() for x in train_not_pay]))

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

In [4]:
prod_view = np.zeros(n_prod+1)
prod_buy = np.zeros(n_prod+1)

In [5]:
n_view = 0
n_buy = 0

for x in train_pay:
    n_view += len(x[0])
    n_buy += len(x[1])
    for y in x[0]:
        prod_view[y] += 1
    for y in x[1]:
        prod_buy[y] += 1
        
for x in train_not_pay:
    n_view += len(x)
    for y in x:
        prod_view[y] += 1
        
prod_view *= (1/n_view)
prod_buy *= (1/n_buy)

2. Реализуйте два алгоритма рекомендаций:

 сортировка просмотренных id по популярности (частота появления в просмотренных),
 
 сортировка просмотренных id по покупаемости (частота появления в покупках).

In [6]:
# работаем только с data_pay
# сразу беру срез
train_pay_rec_view = []
train_pay_rec_buy = []
for l in train_pay:
    train_pay_rec_view.append(sorted(set(l[0]), key = lambda x: prod_view[x], reverse = True)[:5])
    train_pay_rec_buy.append(sorted(set(l[0]), key = lambda x: prod_buy[x], reverse = True)[:5])

In [7]:
test_pay = []
test_not_pay = []
with open("coursera_sessions_test.txt", "r") as f:
    for s in f:
        x = s.rstrip().split(";")
        if not x[1]:
            test_not_pay.append(np.array([int(y) for y in x[0].split(",")]))
        else:
            test_pay.append([np.array([int(y) for y in x[0].split(",")]), np.array([int(y) for y in x[1].split(",")])])

In [8]:
test_pay_rec_view = []
test_pay_rec_buy = []
for l in test_pay:
    test_pay_rec_view.append(sorted(set(l[0]), key = lambda x: prod_view[x], reverse = True)[:5])
    test_pay_rec_buy.append(sorted(set(l[0]), key = lambda x: prod_buy[x], reverse = True)[:5])

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

In [9]:
#сортировка просмотренных id по популярности (частота появления в просмотренных),train
r_1 = np.array([int(y[0] in x[1])/len(x[1]) for (x, y) in zip(train_pay, train_pay_rec_view)]).mean()
p_1 = np.array([int(y[0] in x[1]) for (x, y) in zip(train_pay, train_pay_rec_view)]).mean()
r_5 = np.array([sum(int(k in x[1]) for k in y)/len(x[1]) for (x, y) in zip(train_pay, train_pay_rec_view)]).mean()
p_5 = np.array([sum(int(k in x[1]) for k in y)/5 for (x, y) in zip(train_pay, train_pay_rec_view)]).mean()
print(r_1, p_1, r_5, p_5)

0.44037082805098327 0.5091463414634146 0.8243792477232056 0.2124168514412417


In [11]:
#сортировка просмотренных id по популярности (частота появления в просмотренных),test
r_1 = np.array([int(y[0] in x[1])/len(x[1]) for (x, y) in zip(test_pay, test_pay_rec_view)]).mean()
p_1 = np.array([int(y[0] in x[1]) for (x, y) in zip(test_pay, test_pay_rec_view)]).mean()
r_5 = np.array([sum(int(k in x[1]) for k in y)/len(x[1]) for (x, y) in zip(test_pay, test_pay_rec_view)]).mean()
p_5 = np.array([sum(int(k in x[1]) for k in y)/5 for (x, y) in zip(test_pay, test_pay_rec_view)]).mean()
print(r_1, p_1, r_5, p_5)

0.4156838606292903 0.4796725784447476 0.7981702321435025 0.2037653478854025


In [13]:
#сортировка просмотренных id по покупаемости (частота появления в покупках),train
r_1 = np.array([int(y[0] in x[1])/len(x[1]) for (x, y) in zip(train_pay, train_pay_rec_buy)]).mean()
p_1 = np.array([int(y[0] in x[1]) for (x, y) in zip(train_pay, train_pay_rec_buy)]).mean()
r_5 = np.array([sum(int(k in x[1]) for k in y)/len(x[1]) for (x, y) in zip(train_pay, train_pay_rec_buy)]).mean()
p_5 = np.array([sum(int(k in x[1]) for k in y)/5 for (x, y) in zip(train_pay, train_pay_rec_buy)]).mean()
print(r_1, p_1, r_5, p_5)

0.6776652563376511 0.7935144124168514 0.9274279382233926 0.2529933481152994


In [15]:
#сортировка просмотренных id по покупаемости (частота появления в покупках),test
r_1 = np.array([int(y[0] in x[1])/len(x[1]) for (x, y) in zip(test_pay, test_pay_rec_buy)]).mean()
p_1 = np.array([int(y[0] in x[1]) for (x, y) in zip(test_pay, test_pay_rec_buy)]).mean()
r_5 = np.array([sum(int(k in x[1]) for k in y)/len(x[1]) for (x, y) in zip(test_pay, test_pay_rec_buy)]).mean()
p_5 = np.array([sum(int(k in x[1]) for k in y)/5 for (x, y) in zip(test_pay, test_pay_rec_buy)]).mean()
print(r_1, p_1, r_5, p_5)

0.41054767700622774 0.47394270122783083 0.7914141818379778 0.20163710777626193
